# More Complex Models with the Functional API

Although the Sequential API in Keras is quite straightforward and Sequential models are quite common, it is sometime useful to build networks with more complex topologies, or multiple inputs or outputs. The Keras Functional API provides a more flexible way to define models that makes this possible. It is particularly useful for models with non-linear topology, shared layers, or multiple inputs/outputs. 

In this lecture, we'll delve into the Functional API with a focus on building a ["Wide & Deep" network](https://ai.google/research/pubs/pub45413), a model architecture that can manage both deep and wide feature learning effectively.

#### When to Use the Functional API

- **Model Complexity:** The Functional API is ideal when dealing with complex model architectures, such as those requiring multiple input and output layers, shared layers, or residual connections.
- **Flexibility:** It allows for more customization and control over your model architecture. Unlike the Sequential API, where layers are stacked linearly, the Functional API enables you to define more complex and non-linear relationships.
- **Research and Experimentation:** If you're experimenting with novel architectures or conducting research, the Functional API allows for more experimental flexibility.

## Designing a Wide & Deep Network for More Complex Data

The "Wide & Deep" learning model, conceptualized by Google, is designed to effectively harness both linear and non-linear feature representations. Understanding why and how this model works involves delving into the strengths of its two components: the "wide" part for linear aspects and the "deep" part for complex relationships.

<div style="text-align:center">

![wide and deep nn](assets/wide_and_deep.png)

</div>

#### The "Wide" Component

- **Linear Relationships:**
  - The wide part of the model is designed to learn linear relationships. This means it's good at memorizing or managing sparse data and feature interactions.
  - In practice, this often translates to a form of logistic regression or linear regression, depending on the task.

- **Feature Interactions:**
  - The wide component can effectively capture simple rules and interactions between features. For instance, if certain feature combinations are particularly predictive of the target variable, the wide component can learn this directly.

- **Advantage in Memorization:**
  - The wide part excels in memorization, which is beneficial for learning from sparse data where certain feature combinations may be very predictive but don't occur frequently enough for a deep model to learn and generalize. Categorical features, especially those with a large number of possible values, often work well with the wide part. 

#### The "Deep" Component

- **Complex Relationships:**
  - The deep component, typically a deep neural network, is adept at learning complex, non-linear relationships and feature hierarchies.
  - These networks automatically learn to represent data in a hierarchical fashion (low-level features in early layers, more abstract features in deeper layers).

- **Generalization:**
  - Deep networks are particularly good at generalizing to new scenarios. They can infer unseen relationships based on the hierarchies they've learned, making them powerful for tasks involving unstructured data like images and text.

- **Feature Learning:**
  - Unlike the wide part, deep networks don't require explicit feature engineering. They can automatically learn useful representations from raw data, making them highly versatile and adaptable. Numerical features, particularly those that require complex transformations to relate to the output, are suitable for the deep part.
  

The Wide & Deep model combines the strengths of memorization and generalization. Use the wide part for features where the relationship with the output is expected to be linear or rule-based, and the deep part for more abstract patterns and relationships. For example, in a recommendation system, the wide part can effectively capture specific user-item interactions (like a particular user's preference for a specific genre), while the deep part can learn user preferences and item characteristics from user and item metadata.

When designing a Wide & Deep model, balance is key. Overemphasizing the wide part might lead to overfitting on specific feature combinations, while focusing too much on the deep part might result in missing out on simple, predictive rules in the data. The optimal balance often depends on the specific nature of the dataset and the problem.

## Implementing a Wide & Deep Network

We'll develop a Wide & Deep Network to tackle the [California Housing dataset](https://developers.google.com/machine-learning/crash-course/california-housing-data-description). The dataset includes a mix of features, some of which have a clear linear relationship with the target variable, while others may have more complex, non-linear relationships.  For example, features like 'median income' might have a more straightforward, linear relationship with housing prices. In contrast, features like 'latitude' and 'longitude' could have a non-linear and more complex interaction with the price.

Our goal here is to predict the median house price for each "block group."  

#### Retrieving the data


In [ ]:
import tensorflow as tf
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split

In [5]:
housing = fetch_california_housing()
X_train_full, X_test, y_train_full, y_test = train_test_split(
    housing.data, housing.target, random_state=42)
X_train, X_valid, y_train, y_valid = train_test_split(
    X_train_full, y_train_full, random_state=42)
# Reserve an item for prediction
X_new = X_test[:3]

In [ ]:
print(housing.DESCR)

In [ ]:
import numpy as np
# Note that outputs are in $100,000
print(f"Min {np.min(y_train_full)} - Max {np.max(y_train_full)}")




#### Setting up the model



In [ ]:
# extra code – reset the name counters and make the code reproducible
tf.keras.backend.clear_session()
tf.random.set_seed(42)

# Normalization layer to standardize the inputs,, , and one more Dense layer with a single neuron for the output layer, without any activation function
normalization_layer = tf.keras.layers.Normalization()

# two Dense layers with 30 neurons each, using the ReLU activation function
hidden_layer1 = tf.keras.layers.Dense(30, activation="relu")
hidden_layer2 = tf.keras.layers.Dense(30, activation="relu")

#Concatenate layer - In a Wide & Deep network, you have two paths: the wide path (simple linear relationships) 
#and the deep path (complex non-linear relationships). The concatenate layer is used to merge these two paths. 
#It takes outputs from both the wide and deep components and concatenates them along a specified axis (by default, the last axis).
concat_layer = tf.keras.layers.Concatenate()

#Dense layer with a single neuron for the output layer, without any activation function.  This is the median housing price.
output_layer = tf.keras.layers.Dense(1)

# The Input object (the variable name input_ is used to avoid overshadowing Python’s built-in input() function). 
# This is a specification of the kind of input the model 
# will get, including its shape and optionally its dtype, which defaults to 32-bit floats. 
# A model may actually have multiple inputs, as you will see shortly.
input_ = tf.keras.layers.Input(shape=X_train.shape[1:])


#Normalization layer just like a function, passing it the Input object. This is why this is called the functional API. 
# Note that we are just telling Keras how it should connect the layers together; no actual data is being processed yet, 
# as the Input object is just a data specification. In other words, it’s a symbolic input. The output of this call is 
# also symbolic: normalized doesn’t store any actual data, it’s just used to construct the model.
normalized = normalization_layer(input_)

# In the same way, we then pass normalized to hidden_layer1, which outputs hidden1, 
# and we pass hidden1 to hidden_layer2, which outputs hidden2.
hidden1 = hidden_layer1(normalized)
hidden2 = hidden_layer2(hidden1)

#So far we’ve connected the layers sequentially, but then we use the concat_layer to 
# concatenate the input and the second hidden layer’s output. Again, no actual data is 
# concatenated yet: it’s all symbolic, to build the model.
concat = concat_layer([normalized, hidden2])

# Then we pass concat to the output_layer, which gives us the final output.
output = output_layer(concat)

# Finally create the model, specifying inputs and outputs
model = tf.keras.Model(inputs=[input_], outputs=[output])
model.summary()

Once again, we compile and train the model, just like before...

In [ ]:
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)
model.compile(loss="mse", optimizer=optimizer, metrics=["RootMeanSquaredError"])

# This is necessary in order to make sure the normalization layer correctly calculate things like the mean and 
# standard deviation necessary
normalization_layer.adapt(X_train)
history = model.fit(X_train, y_train, epochs=20,
                    validation_data=(X_valid, y_valid))
mse_test = model.evaluate(X_test, y_test)
y_pred = model.predict(X_new)

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

pd.DataFrame(history.history).plot(
    figsize=(8, 5), xlim=[0, 20], ylim=[0, 2], grid=True, xlabel="Epoch",
    style=["r--", "r--.", "b-", "b-*"])
plt.legend(loc="lower left")  # extra code
plt.show()

Yikes!  Note that an RMS error of 1 (roughly our mean validation error) is $100,000, which isn't great.

There are a few things we might try.  First, we might consider splitting our features, so that the "wide" and "deep" part of the network focus on different subsets of features.  For example, we might send the first five features through the wide path (based on the assumption that these should have a linear relationship with price) and the last six features through the deep path (assuming a more complex relationship here). Note that this is how we can use our domain knowledge to construct a network!

The envisioned network looks something like:

<div style="text-align:center">

![Wide and deep 2](assets/wide_and_deep2.png)

</div>


In [10]:
tf.keras.backend.clear_session()
tf.random.set_seed(42)

input_wide = tf.keras.layers.Input(shape=[5]) # features 0 to 4 
input_deep = tf.keras.layers.Input(shape=[6]) # features 2 to 7 
norm_layer_wide = tf.keras.layers.Normalization() 
norm_layer_deep = tf.keras.layers.Normalization()
norm_wide = norm_layer_wide(input_wide)
norm_deep = norm_layer_deep(input_deep)

# Note, the following is just a shorthand where we construct and then attach the network all in one go
hidden1 = tf.keras.layers.Dense(30, activation="relu")(norm_deep)
hidden2 = tf.keras.layers.Dense(30, activation="relu")(hidden1)
concat = tf.keras.layers.concatenate([norm_wide, hidden2])
output = tf.keras.layers.Dense(1)(concat)
model = tf.keras.Model(inputs=[input_wide, input_deep], outputs=[output])

Now we compile as usual, but we need to pass a pair of matrices in order to align with the expected input, and similarly with the validation and test data.

In [ ]:
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)
model.compile(loss="mse", optimizer=optimizer, metrics=["RootMeanSquaredError"])

# Note the offsets are used here to split up the features according to our input layer
X_train_wide, X_train_deep = X_train[:, :5], X_train[:, 2:] 
X_valid_wide, X_valid_deep = X_valid[:, :5], X_valid[:, 2:] 
X_test_wide, X_test_deep = X_test[:, :5], X_test[:, 2:] 
X_new_wide, X_new_deep = X_test_wide[:3], X_test_deep[:3]


norm_layer_wide.adapt(X_train_wide) 
norm_layer_deep.adapt(X_train_deep)

history = model.fit((X_train_wide, X_train_deep), y_train, epochs=20, validation_data=((X_valid_wide, X_valid_deep), y_valid)) 
mse_test = model.evaluate((X_test_wide, X_test_deep), y_test)
y_pred = model.predict((X_new_wide, X_new_deep))

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

pd.DataFrame(history.history).plot(
    figsize=(8, 5), xlim=[0, 20], ylim=[0, 2], grid=True, xlabel="Epoch",
    style=["r--", "r--.", "b-", "b-*"])
plt.legend(loc="lower left")  # extra code
plt.show()

Yikes! Another thing we can try here is to give the hidden layers an explicit training signal in order to force them to learn something directly about the data.  This is a type of regularization, because it helps to constrain the hidden layers so that they are not simply "memorizing" the data.  Our new architecture would look something like:

<div style="text-align: center">

![Wide and deep 3](assets/wide_and_deep3.png)

</div>

Our auxiliary output requires is own loss function, but these two losses are combined during training.  Keras allows us to specify a weighting that will allow us to control the relative importance of these two losses.

In [13]:
tf.keras.backend.clear_session()
tf.random.set_seed(42)

input_wide = tf.keras.layers.Input(shape=[5])  # features 0 to 4
input_deep = tf.keras.layers.Input(shape=[6])  # features 2 to 7
norm_layer_wide = tf.keras.layers.Normalization()
norm_layer_deep = tf.keras.layers.Normalization()
norm_wide = norm_layer_wide(input_wide)
norm_deep = norm_layer_deep(input_deep)
hidden1 = tf.keras.layers.Dense(30, activation="relu")(norm_deep)
hidden2 = tf.keras.layers.Dense(30, activation="relu")(hidden1)
concat = tf.keras.layers.concatenate([norm_wide, hidden2])
output = tf.keras.layers.Dense(1)(concat)
aux_output = tf.keras.layers.Dense(1)(hidden2)
model = tf.keras.Model(inputs=[input_wide, input_deep],
                       outputs=[output, aux_output])

Now we compile model and specify the different loss functions.

In [14]:
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)
model.compile(loss=("mse", "mse"), loss_weights=(0.9, 0.1), optimizer=optimizer,
              metrics=["RootMeanSquaredError"])

Now, training, being sure to specify the output labels for both output layers.

In [ ]:
# Remember to adapt the normalization layers!!!
norm_layer_wide.adapt(X_train_wide)
norm_layer_deep.adapt(X_train_deep)
history = model.fit(
    (X_train_wide, X_train_deep), (y_train, y_train), epochs=20,
    validation_data=((X_valid_wide, X_valid_deep), (y_valid, y_valid))
)

In [ ]:
eval_results = model.evaluate((X_test_wide, X_test_deep), (y_test, y_test))
weighted_sum_of_losses, main_loss, aux_loss, main_rmse, aux_rmse = eval_results
print(f"""
Weighed Sum of Losses: {weighted_sum_of_losses}
Main Loss: {main_loss}
Aux Loss: {aux_loss}
Main RMSE: {main_rmse}
Aux RMSE: {aux_rmse}
      """)

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

pd.DataFrame(history.history).plot(
    figsize=(8, 5), xlim=[0, 20], ylim=[0, 2], grid=True, xlabel="Epoch",
    style=["r--", "r--.", "b-", "b-*"])
plt.legend(loc="lower left")  # extra code
plt.show()

Note that this appears to have substantially improved things for the validation set, indicating that we indeed seem to be generalizing our data quite a bit better!

## Additional Software Features

Keras provides several additional software features that can be used to make the process of working with networks more useful.

#### The Subclassing API

The Subclassing API allows you to define custom models by subclassing the `tf.keras.Model` class. In this approach, you create a new class that inherits from `tf.keras.Model` and defines your own forward pass for the model. This method provides the greatest flexibility in defining models, as you can add loops, conditional branches, and even custom layers directly within your model.

#### Why Use the Subclassing API?

1. **Custom Behavior:**
   - If you need a model that does not fit the standard sequential or functional paradigms, subclassing enables you to implement custom behavior in the forward pass.

2. **Research and Experimentation:**
   - For researchers experimenting with new ideas, subclassing offers a way to express complex models or layers in an intuitive manner.

3. **Dynamic Models:**
   - Subclassing is particularly suited for dynamic models, where the architecture can change based on the input data, something that's harder to achieve with the Sequential and Functional APIs.

#### How to Implement a Model using Subclassing?

1. **Define a Model Class:**
   - Create a new class that inherits from `tf.keras.Model`.
   - Define the layers in the `__init__` method.
   - Implement the model's forward pass in the `call` method.

2. **Instantiate and Use the Model:**
   - Like other Keras models, you compile and train it using `.compile()` and `.fit()` methods.
   - It can be used for prediction with `.predict()`.

#### Key Points

- **Debugging:** Subclassing can make debugging easier since it works just like regular Python code.
- **Complexity:** It's slightly more complex and error-prone than using Sequential or Functional APIs due to the lack of model validation before training.
- **Serialization:** Subclassed models have limitations in terms of saving and loading. The structure of the model is not explicitly defined, so when saving the model, Keras saves the model's weights only.


In [17]:
class WideAndDeepModel(tf.keras.Model):
    def __init__(self, units=30, activation="relu", **kwargs):
        super().__init__(**kwargs)  # needed to support naming the model
        self.norm_layer_wide = tf.keras.layers.Normalization()
        self.norm_layer_deep = tf.keras.layers.Normalization()
        self.hidden1 = tf.keras.layers.Dense(units, activation=activation)
        self.hidden2 = tf.keras.layers.Dense(units, activation=activation)
        self.main_output = tf.keras.layers.Dense(1)
        self.aux_output = tf.keras.layers.Dense(1)
        
    def call(self, inputs):
        input_wide, input_deep = inputs
        norm_wide = self.norm_layer_wide(input_wide)
        norm_deep = self.norm_layer_deep(input_deep)
        hidden1 = self.hidden1(norm_deep)
        hidden2 = self.hidden2(hidden1)
        concat = tf.keras.layers.concatenate([norm_wide, hidden2])
        output = self.main_output(concat)
        aux_output = self.aux_output(hidden2)
        return output, aux_output

tf.random.set_seed(42)  # extra code – just for reproducibility
model = WideAndDeepModel(30, activation="relu", name="my_cool_model")

Now, to instantiate and train:

In [ ]:
tf.keras.backend.clear_session()
tf.random.set_seed(42)  # extra code – just for reproducibility
model = WideAndDeepModel(30, activation="relu", name="my_cool_model")
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)
model.compile(loss="mse", loss_weights=[0.9, 0.1], optimizer=optimizer,
              metrics=["RootMeanSquaredError"])
model.norm_layer_wide.adapt(X_train_wide)
model.norm_layer_deep.adapt(X_train_deep)
history = model.fit(
    (X_train_wide, X_train_deep), (y_train, y_train), epochs=10,
    validation_data=((X_valid_wide, X_valid_deep), (y_valid, y_valid)))
eval_results = model.evaluate((X_test_wide, X_test_deep), (y_test, y_test))
weighted_sum_of_losses, main_loss, aux_loss, main_rmse, aux_rmse = eval_results
y_pred_main, y_pred_aux = model.predict((X_new_wide, X_new_deep))

## Saving and restoring models

Models can be easily saved off for future use with:

In [ ]:
model.save("my_keras_model", save_format="tf")

All aspects of the model are saved this way, including weights and any hyperparameters.  Loading the model can be easily accomplished with:

In [ ]:
model = tf.keras.models.load_model("my_keras_model")

You can also save and load just the weights for a network, which is faster uses less disk space then storing the entire model.  This is useful when saving "checkpoints" during the model training process.  Checkpoints provide a restore point if something goes awry during the training process.  To accomplish this, we use callbacks, described below. 

### Using Callbacks

Sometimes it is useful to interrupt the training process to inspect aspects of the network. The `fit()` methods accepts a set of functions (referred to as callbacks) which can be run during the training process.  There are three places when you can run callback functions:
- Before and after training
- Before and after each epoch
- Before and after each batch

Keras offers a number of [pre-built callbacks](https://keras.io/api/callbacks/) and you [can write your own](https://keras.io/guides/writing_your_own_callbacks/) if you want.

#### Examples

**Model Checkpoints**

The ModelCheckpoint callback allows you to save checkpoints at regular intervals during training.  This can be useful both for inspection, and as a backup mechanism in case something goes wrong during training. This is also a nice way to avoid overtraining with too many epochs, since it is easy to roll back to a previous training state.

~~~python
checkpoint_cb = tf.keras.callbacks.ModelCheckpoint("my_checkpoints", save_weights_only=True)
history = model.fit([...], callbacks=[checkpoint_cb])
~~~

**Early Stopping**

The `EarlyStopping` callback allows you to interrupt training when no progress is made on the validation step for a number of epochs.  Setting `restore_best_weights=True` will roll back to the best model at the end of training.

~~~python
early_stopping_cb = tf.keras.callbacks.EarlyStopping(patience=10,restore_best_weights=True)
history = model.fit([...], callbacks=[checkpoint_cb, early_stopping_cb])
~~~

### Using Tensorboard

Tensorflow provides a UI called Tensorboard for inspecting the performance of training.  To use Tensorboard, we need to log data in a special format using the Tensorboard callback.  Generally, speaking, it's useful to organize log data with a single top level directory for all logging (you might want to add this to your `.gitignore` file) and then create logs for different runs in subdirectories.  The following code takes care of creating the path where logs should reside.

In [ ]:
from pathlib import Path 
from time import strftime
def get_run_logdir(root_logdir="./logs"):
    return Path(root_logdir) / strftime("run_%Y_%m_%d_%H_%M_%S")


get_run_logdir()

To use Tensorboard, all we need is to install the callback when we train the model.  First, set up the model (we'll use the subclassed version above):

In [20]:
tf.keras.backend.clear_session()
tf.random.set_seed(42)  # extra code – just for reproducibility
model = WideAndDeepModel(30, activation="relu", name="my_cool_model")
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)
model.compile(loss="mse", loss_weights=[0.9, 0.1], optimizer=optimizer,
              metrics=["RootMeanSquaredError"])
model.norm_layer_wide.adapt(X_train_wide)
model.norm_layer_deep.adapt(X_train_deep)

Then, we'll add the tensorboard callback and train.

In [ ]:
test_dir = get_run_logdir()
tensorboard_cb = tf.keras.callbacks.TensorBoard(test_dir, profile_batch=(100, 200))
history=model.fit(
    (X_train_wide, X_train_deep), (y_train, y_train), epochs=20,
    validation_data=((X_valid_wide, X_valid_deep), (y_valid, y_valid)),callbacks=[tensorboard_cb])

Now, for comparison's sake, we'll run the model again with a slightly higher learning rate.

In [23]:
tf.keras.backend.clear_session()
tf.random.set_seed(42)  # extra code – just for reproducibility
model = WideAndDeepModel(30, activation="relu", name="my_cool_model")

# Doubled the learning
optimizer = tf.keras.optimizers.Adam(learning_rate=2e-3)
model.compile(loss="mse", loss_weights=[0.9, 0.1], optimizer=optimizer,
              metrics=["RootMeanSquaredError"])
model.norm_layer_wide.adapt(X_train_wide)
model.norm_layer_deep.adapt(X_train_deep)

In [ ]:
test_dir = get_run_logdir()
tensorboard_cb = tf.keras.callbacks.TensorBoard(test_dir, profile_batch=(100, 200))
history=model.fit(
    (X_train_wide, X_train_deep), (y_train, y_train), epochs=20,
    validation_data=((X_valid_wide, X_valid_deep), (y_valid, y_valid)),callbacks=[tensorboard_cb])

Now we can start up `tensorboard` to have a look at our log files.  Once tensorboard is running, you can browse to http://localhost:6006/ to have a look.  

In [ ]:
%load_ext tensorboard 
%tensorboard --logdir=./logs

Tensorboard is a highly functional tool, and has some neat functions for visualizing high dimensional embedding spaces within a network.  You can read more about this [here](https://www.tensorflow.org/tensorboard/tensorboard_projector_plugin).